In [ ]:
pip install requests bs4 pandas firebase-admin

In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import csv
import json

Initialize Firestore DB

In [3]:
#load in firestore credentials
# https://firebase.google.com/docs/firestore/quickstart
# DON"T FORGET TO DELETE THE PATH WHEN PUSHING TO GITHUB!!!!
cred = credentials.Certificate('PATH_TO_FIRESTORE_KEY')
app = firebase_admin.initialize_app(cred)
db = firestore.client()

This is what I would be storing for each twit

In [7]:
twit_url = "https://stocktwits.com/user/message/497084294"
print('url: ', twit_url)
page = requests.get(twit_url)
soup = BeautifulSoup(page.content, 'html.parser')
twit_text = str(soup.head.find("meta", attrs={"name" : "description"})["content"])
print('text: ', twit_text)
twit_user = str(soup.head.find("meta", attrs={"name" : "twitter:title"})["content"]).replace(' | Stocktwits','')
print('user: ', twit_user)
print('userUrl: https://stocktwits.com/'+twit_user.replace('@',''))

url:  https://stocktwits.com/user/message/497084294
text:  $SAVA stop loss raid this afternoon was great buying opportunity loaded shares under $30!
user:  $SAVA stop loss raid this afternoon was great buying opportunity loaded shares under $30!
userUrl: https://stocktwits.com/$SAVA stop loss raid this afternoon was great buying opportunity loaded shares under $30!


In [23]:
twit_url = "https://stocktwits.com/user/message/497084294"
print('url: ', twit_url)
page = requests.get(twit_url)
soup = BeautifulSoup(page.content, 'html.parser')
twit_text = soup.body.find("script", attrs={"id" : "__NEXT_DATA__"}).string
twit_json = json.loads(twit_text)
# print(twit_json)
print(twit_json['props']['pageProps']['initialData']['message'])
twit_text = str(soup.head.find("meta", attrs={"name" : "description"})["content"])
print('text: ', twit_text)
twit_user = str(soup.head.find("meta", attrs={"name" : "twitter:title"})["content"]).replace(' | Stocktwits','')
print('user: ', twit_user)
print('userUrl: https://stocktwits.com/'+twit_user.replace('@',''))

url:  https://stocktwits.com/user/message/497084294
{'id': 497084294, 'body': '$SAVA stop loss raid this afternoon was great buying opportunity loaded shares under $30!', 'created_at': '2022-11-18T21:17:30Z', 'user': {'id': 3011634, 'username': 'CaymanSlushFund', 'name': 'Cayman Slush Fund', 'avatar_url': 'https://avatars.stocktwits.com/production/3011634/thumb-1578414526.png', 'avatar_url_ssl': 'https://avatars.stocktwits.com/production/3011634/thumb-1578414526.png', 'join_date': '2020-01-07', 'official': False, 'identity': 'User', 'classification': [], 'search_regions': [], 'followers': 296, 'following': 473, 'ideas': 1935, 'watchlist_stocks_count': 178, 'like_count': 946, 'plus_tier': '', 'premium_room': '', 'trade_app': False, 'trade_status': 'PENDING_MFA', 'subscribers_count': 39, 'subscribed_to_count': 55, 'following_stocks': 0, 'location': '', 'bio': None, 'website_url': None, 'trading_strategy': {'assets_frequently_traded': ['Equities', 'Options'], 'approach': 'Growth', 'holdin

Example on how to get data - get our index flag for last message scraped

In [34]:
# we would first start by retrieving the last good message id
doc_ref = db.collection(u'twit').document(u'startIndex')
# https://firebase.google.com/docs/firestore/query-data/get-data
doc = doc_ref.get()
if doc.exists:
    print(f'Document data: {doc.to_dict()}')
else:
    print(u'No such document!')

Document data: {'testIndex': 1, 'index': 0}


Example on how to push data - updating our index flag

In [35]:
data = {
    u'testIndex': 10,
}
# https://firebase.google.com/docs/firestore/manage-data/add-data
# Updating our doc_ref from above to whatever we specify in data.index
doc_ref.set(data, merge=True)
# Check if it worked by replaying above get data example and see if it changed

update_time {
  seconds: 1668814840
  nanos: 26139000
}

Next steps for creating the actual scraper:
1. Loop through starting at index
2. https://stocktwits.com/message/# -> # should be the index
3. Use bs4 and add current data to csv as new row
4. End loop when no more new messages ~ will need to talk with team on how to determine this
5. Update index to what first bad index was
6. Push csv to storage - I will add this code in soon
7. End

In [10]:
#this is example
headers = ['id', 'twit_url', 'twit_text', 'twit_user', 'twit_user_url']
data = []
#change range for more samples if needed
for i in range(250):
    twit_url = "https://stocktwits.com/message/" + str(i)
    page = requests.get(twit_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    twit_text = str(soup.head.find("meta", attrs={"name" : "description"})["content"]).replace(',','')
    twit_user = str(soup.head.find("meta", attrs={"name" : "twitter:title"})["content"]).replace(' | Stocktwits','')
    twit_user_url = 'https://stocktwits.com/'+twit_user.replace('@','')
    if twit_user != '@':
        data.append([i, twit_url, twit_text, twit_user, twit_user_url])
#change name if wanting multiple example csvs
with open('../csv/example.csv', 'w+') as file:
    writer = csv.writer(file)
    writer.writerow(headers)
    writer.writerows(data)

In [28]:
import json
import requests
import google.oauth2.id_token
import google.auth.transport.requests
import urllib
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./cred.json"

request = google.auth.transport.requests.Request()
audience = 'https://us-central1-cs410-7733e.cloudfunctions.net/function-1'
TOKEN = google.oauth2.id_token.fetch_id_token(request, audience)

r = requests.post(
    'https://us-central1-cs410-7733e.cloudfunctions.net/function-1', 
    headers={'Authorization': f"Bearer {TOKEN}", "Content-Type": "application/json"},
    data=json.dumps({"query": "$ROYL"})  # possible request parameters
)
print(r.content)

b'[\n    {\n        "id": 239,\n        "twit_url": "https://stocktwits.com/message/239",\n        "twit_text": "still having fun with $ROYL and $SPF",\n        "twit_user": "@whatthetrading",\n        "twit_user_url": "https://stocktwits.com/whatthetrading",\n        "score": 4.2066119468\n    },\n    {\n        "id": 249,\n        "twit_url": "https://stocktwits.com/message/249",\n        "twit_text": "$ROYL having a tough time in the premarket also",\n        "twit_user": "@whatthetrading",\n        "twit_user_url": "https://stocktwits.com/whatthetrading",\n        "score": 3.8736441945\n    },\n    {\n        "id": 245,\n        "twit_url": "https://stocktwits.com/message/245",\n        "twit_text": "reserved shares of $SPDE $ROYL and $GRO as possible shorts for later today",\n        "twit_user": "@timothysykes",\n        "twit_user_url": "https://stocktwits.com/timothysykes",\n        "score": 3.3442297092\n    }\n]'
